# Experiment 01: Airline dataset

In this notebook we are going to perform a prediction experiment with XGBoost and LightGBM using the Airline dataset. We are also going to analyze if the dataset has [concept drift](https://en.wikipedia.org/wiki/Concept_drift). 

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

The machine we used was an [Azure NV24 VM](https://azure.microsoft.com/en-gb/blog/azure-n-series-general-availability-on-december-1/), which has [NVIDIA M60](http://www.nvidia.com/object/tesla-m60.html) GPUs. Its operating system is Ubuntu 16.04.

For both XGBoost and LightGBM we compiled from source, to get the last improvements. In XGboost we used the commit `6776292951565c8cd72e69afd9d94de1474f00c0` of May 26th. For LightGBM we used the commit `73968a96829e212b333c88cd44725c8c39c03ad1` of June 2nd. To get these versions and replicate our experiments:
```python
git clone --recursive *url_of_library*
git checkout *oldcommit*
git submodule update --recursive
```

In [1]:
import json
import sys
import warnings

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pkg_resources
from IPython.display import SVG, display
from bokeh.io import export_svgs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.loaders import load_airline
from libs.notebook_memory_management import start_watching_memory
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, precision_score,
                             recall_score)
from libs.metrics import classification_metrics_binary, classification_metrics_binary_prob, binarize_prediction
from toolz import curry
from xgboost import XGBRegressor

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning)

Using TensorFlow backend.


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


The dataset we are going to use in this notebook is huge, therefore we want to monitor the memory consumption. 

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
start_watching_memory()

In [3] used 12.0430 MiB RAM in 18.05s, total RAM usage 271.04 MiB


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [4]:
%%time
df_plane = load_airline()
print(df_plane.shape)

INFO:libs.loaders:MOUNT_POINT not found in environment. Defaulting to /fileshare


(115069017, 14)
CPU times: user 1min 29s, sys: 18.9 s, total: 1min 48s
Wall time: 4min 56s
In [4] used 21994.3711 MiB RAM in 296.71s, total RAM usage 22265.41 MiB


In [5]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [5] used 0.2305 MiB RAM in 0.12s, total RAM usage 22265.64 MiB


The first step is to convert the categorical features to numeric features.

In [6]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 34s, sys: 13.6 s, total: 1min 48s
Wall time: 1min 48s
In [6] used 5270.4883 MiB RAM in 108.91s, total RAM usage 27536.13 MiB


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [7] used 0.0039 MiB RAM in 0.12s, total RAM usage 27536.13 MiB


In [8]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 51.4 s, sys: 10.6 s, total: 1min 1s
Wall time: 1min 2s
In [8] used 13168.8477 MiB RAM in 62.48s, total RAM usage 40704.98 MiB


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [9] used 0.0117 MiB RAM in 0.11s, total RAM usage 40704.99 MiB


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [10]:
df_plane_numeric = df_plane_numeric.apply(lambda x: x.astype('int16'))

In [10] used 3072.6328 MiB RAM in 12.85s, total RAM usage 43777.62 MiB


In [11]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 296 ms, sys: 376 ms, total: 672 ms
Wall time: 674 ms
In [11] used 877.9766 MiB RAM in 0.81s, total RAM usage 44655.60 MiB


In [12]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [12] used 0.0000 MiB RAM in 0.11s, total RAM usage 44655.60 MiB


Once the features are prepared, let's split the dataset into train and test set. We won't use validation for this example (however, you can try to add it).

In [13]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [13] used 0.0000 MiB RAM in 0.11s, total RAM usage 44655.60 MiB


In [14]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(92055213, 15)
(0, 15)
(23013804, 15)
CPU times: user 34.7 s, sys: 3.46 s, total: 38.1 s
Wall time: 38.3 s
In [14] used 4653.3594 MiB RAM in 38.45s, total RAM usage 49308.96 MiB


In [15]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [15] used 0.0039 MiB RAM in 0.10s, total RAM usage 49308.96 MiB


In [16]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)

CPU times: user 768 ms, sys: 708 ms, total: 1.48 s
Wall time: 1.51 s
In [16] used 2853.1641 MiB RAM in 1.61s, total RAM usage 52162.13 MiB


In [17]:
del train, validate, test

In [17] used 0.0000 MiB RAM in 0.10s, total RAM usage 52162.13 MiB


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [18]:
results_dict = dict()

In [18] used 0.0000 MiB RAM in 0.10s, total RAM usage 52162.13 MiB


In [19]:
number_processors = get_number_processors()
print(number_processors)

24
In [19] used 0.0000 MiB RAM in 0.10s, total RAM usage 52162.13 MiB


Let's start with the XGBoost classifier.

In [20]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=50,
                                min_child_weight=30,
                                learning_rate=0.1,
                                colsample_bytree=0.80,
                                scale_pos_weight=2,
                                gamma=0.1,
                                reg_lambda=1,
                                subsample=1,
                                n_jobs=number_processors,
                                random_state=77)

In [20] used 0.1016 MiB RAM in 0.10s, total RAM usage 52162.23 MiB


In [21]:
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train)

In [21] used 20403.8633 MiB RAM in 7268.58s, total RAM usage 72566.09 MiB


In [22]:
results_dict['xgb']={
    'train_time': t.interval
}

In [22] used 0.0000 MiB RAM in 0.10s, total RAM usage 72566.09 MiB


Training XGBoost model with leave-wise growth

In [23]:
xgb_hist_clf_pipeline = XGBRegressor(max_depth=0,
                                    n_estimators=50,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    colsample_bytree=0.80,
                                    scale_pos_weight=2,
                                    gamma=0.1,
                                    reg_lambda=1,
                                    subsample=1,
                                    max_leaves=255,
                                    grow_policy='lossguide',
                                    tree_method='hist',
                                    n_jobs=number_processors,
                                    random_state=77)

In [23] used 0.0000 MiB RAM in 0.10s, total RAM usage 72566.09 MiB


In [24]:
with Timer() as t:
    xgb_hist_clf_pipeline.fit(X_train, y_train)

In [24] used 23254.1719 MiB RAM in 416.74s, total RAM usage 95820.27 MiB


In [25]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

In [25] used 0.0078 MiB RAM in 0.10s, total RAM usage 95820.27 MiB


Training LightGBM model

In [26]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=50,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  colsample_bytree=0.80,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  nthread=number_processors,
                                  seed=77)

In [26] used 0.0000 MiB RAM in 0.10s, total RAM usage 95820.27 MiB


In [27]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

In [27] used 5463.3906 MiB RAM in 340.69s, total RAM usage 101283.66 MiB


In [28]:
results_dict['lgbm']={
    'train_time': t.interval
}

In [28] used -2282.5586 MiB RAM in 0.11s, total RAM usage 99001.11 MiB


As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [29]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [29] used 180.3672 MiB RAM in 10.28s, total RAM usage 99181.47 MiB


In [30]:
results_dict['xgb']['test_time'] = t.interval

In [30] used 0.0039 MiB RAM in 0.10s, total RAM usage 99181.48 MiB


In [31]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [31] used 87.7930 MiB RAM in 11.72s, total RAM usage 99269.27 MiB


In [32]:
results_dict['xgb_hist']['test_time'] = t.interval

In [32] used 0.0078 MiB RAM in 0.10s, total RAM usage 99269.28 MiB


In [33]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [33] used 746.2188 MiB RAM in 12.16s, total RAM usage 100015.50 MiB


In [34]:
results_dict['lgbm']['test_time'] = t.interval

In [34] used 0.0000 MiB RAM in 0.10s, total RAM usage 100015.50 MiB


### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [35]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)

In [35] used 526.6250 MiB RAM in 0.52s, total RAM usage 100542.12 MiB


In [36]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

In [36] used 517.6094 MiB RAM in 26.92s, total RAM usage 101059.73 MiB


In [37]:
results_dict['xgb']['performance'] = report_xgb

In [37] used 0.0039 MiB RAM in 0.10s, total RAM usage 101059.73 MiB


In [38]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [38] used 14.1133 MiB RAM in 23.85s, total RAM usage 101073.85 MiB


In [39]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [39] used 0.0039 MiB RAM in 0.10s, total RAM usage 101073.85 MiB


In [40]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [40] used 0.0000 MiB RAM in 25.18s, total RAM usage 101073.85 MiB


In [41]:
results_dict['lgbm']['performance'] = report_lgbm

In [41] used 0.0000 MiB RAM in 0.10s, total RAM usage 101073.85 MiB


In [42]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "AUC": 0.8089440012596557,
            "Accuracy": 0.7357180064625561,
            "F1": 0.6941205450307804,
            "Precision": 0.7701217434739469,
            "Recall": 0.6317726394571114
        },
        "test_time": 11.95346931399763,
        "train_time": 337.04530743800206
    },
    "xgb": {
        "performance": {
            "AUC": 0.7850011114714786,
            "Accuracy": 0.6293210805132433,
            "F1": 0.6976245180307368,
            "Precision": 0.5691896488110514,
            "Recall": 0.9009106033700574
        },
        "test_time": 10.075740259999293,
        "train_time": 7197.378043801
    },
    "xgb_hist": {
        "performance": {
            "AUC": 0.807842644202481,
            "Accuracy": 0.6732816964983277,
            "F1": 0.716900958176553,
            "Precision": 0.6088530428953446,
            "Recall": 0.8715712075869363
        },
        "test_time": 11.461563738001132,
        "tra

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

In [43]:
del xgb_clf_pipeline, xgb_hist_clf_pipeline, lgbm_clf_pipeline, X_train, X_test, X_val

In [43] used -48613.8477 MiB RAM in 0.78s, total RAM usage 52460.00 MiB


# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [44]:
initial_year = 1987
num_ini = 5

In [44] used 0.0000 MiB RAM in 0.10s, total RAM usage 52460.00 MiB


In [45]:
def generate_subset_by_year(df, year_ini, year_end):
    return df[df['Year'].isin(range(year_ini, year_end))]

In [45] used 0.0000 MiB RAM in 0.10s, total RAM usage 52460.00 MiB


In [46]:
%%time
subset_base = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + num_ini)
print(subset_base.shape)

(16810190, 15)
CPU times: user 35.7 s, sys: 2.61 s, total: 38.3 s
Wall time: 38.5 s
In [46] used 448.8867 MiB RAM in 38.64s, total RAM usage 52908.89 MiB


In [47]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_base.index)]
print(rest_df.shape)

(98258827, 15)
CPU times: user 52.6 s, sys: 4.64 s, total: 57.2 s
Wall time: 57.9 s
In [47] used 10267.1094 MiB RAM in 57.96s, total RAM usage 63176.00 MiB


### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [48]:
X_train, y_train = generate_feables(subset_base)
del(subset_base)

In [48] used 416.8203 MiB RAM in 0.31s, total RAM usage 63592.82 MiB


In [49]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [49] used 0.0000 MiB RAM in 0.10s, total RAM usage 63592.82 MiB


In [50]:
%%time
clf.fit(X_train, y_train)

CPU times: user 27min 54s, sys: 54.9 s, total: 28min 48s
Wall time: 1min 17s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [50] used -23983.4883 MiB RAM in 77.80s, total RAM usage 39609.33 MiB


### Prediction

In [51]:
@curry
def predict_accuracy(clf, test_df):
    X_test, y_test = generate_feables(test_df)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [51] used 0.0000 MiB RAM in 0.11s, total RAM usage 39609.33 MiB


In [52]:
%%time
accuracy_series = rest_df.groupby('Year').apply(predict_accuracy(clf))
print(accuracy_series)

Year
1992    0.757375
1993    0.756093
1994    0.744296
1995    0.731503
1996    0.722490
1997    0.720751
1998    0.706178
1999    0.700762
2000    0.689513
2001    0.674223
2002    0.680176
2003    0.686131
2004    0.680697
2005    0.673467
2006    0.663597
2007    0.651520
2008    0.636613
dtype: float64
CPU times: user 35min 20s, sys: 17.4 s, total: 35min 37s
Wall time: 1min 59s
In [52] used 2624.0000 MiB RAM in 119.96s, total RAM usage 42233.33 MiB


From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [53]:
new_init = 15

In [53] used 0.0000 MiB RAM in 0.10s, total RAM usage 42233.33 MiB


In [54]:
%%time
subset_retrain = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + new_init) 
print(subset_retrain.shape)

(69425349, 15)
CPU times: user 1min 57s, sys: 3.61 s, total: 2min 1s
Wall time: 2min 2s
In [54] used 1853.8594 MiB RAM in 122.20s, total RAM usage 44087.19 MiB


In [55]:
X_train, y_train = generate_feables(subset_retrain)

In [55] used 1304.6250 MiB RAM in 0.98s, total RAM usage 45391.82 MiB


In [56]:
clf_retrain = LGBMClassifier(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

In [56] used 0.0000 MiB RAM in 0.10s, total RAM usage 45391.82 MiB


In [57]:
%%time
clf_retrain.fit(X_train, y_train)

CPU times: user 1h 42min 36s, sys: 2min 58s, total: 1h 45min 34s
Wall time: 4min 45s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [57] used 4105.0039 MiB RAM in 286.01s, total RAM usage 49496.82 MiB


### Prediction

In [58]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_retrain.index)]
print(rest_df.shape)

(45643668, 15)
CPU times: user 32.6 s, sys: 2.86 s, total: 35.4 s
Wall time: 34 s
In [58] used -1695.8203 MiB RAM in 34.12s, total RAM usage 47801.00 MiB


In [59]:
%%time
accuracy_retrain = rest_df.groupby('Year').apply(predict_accuracy(clf_retrain))
print(accuracy_retrain)

Year
2002    0.749985
2003    0.748284
2004    0.724928
2005    0.719791
2006    0.707446
2007    0.695424
2008    0.697949
dtype: float64
CPU times: user 14min 19s, sys: 33.7 s, total: 14min 53s
Wall time: 51.8 s
In [59] used -318.1523 MiB RAM in 51.98s, total RAM usage 47482.85 MiB


### Plot

In [60]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1, color='#5975a4')
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2, color='#5f9e6f')
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    return ax

In [60] used 0.0000 MiB RAM in 0.11s, total RAM usage 47482.85 MiB


In [61]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool

In [61] used 0.0000 MiB RAM in 0.10s, total RAM usage 47482.85 MiB


In [62]:
data_cds = ColumnDataSource(pd.DataFrame({
    'train_acc': accuracy_series,
    'retrain_acc': accuracy_retrain
}))

In [62] used 0.1094 MiB RAM in 0.26s, total RAM usage 47482.96 MiB


In [63]:
# Airline Retrain Results
p = figure(y_axis_label='Accuracy', plot_width=700, plot_height=350, tools="pan,wheel_zoom,box_zoom,reset")
l1 = p.line('Year', 'train_acc', legend=' Train AUC', line_color="#5975a4", source=data_cds, line_width=6, line_cap="round")
p.line('Year', 'retrain_acc', legend=' Retrain AUC', line_color="#a1bae3", source=data_cds, line_width=6, line_cap="round")
l1_hover = HoverTool(renderers=[l1], tooltips=[( 'Train',  '@{train_acc}{0.4f}' ), ( 'Retrain',  '@{retrain_acc}{0.4f}' )], mode='vline')
p.add_tools(l1_hover)
show(p)

In [63] used 0.0000 MiB RAM in 0.19s, total RAM usage 47482.96 MiB


In [64]:
# SVG: Airline Retrain Results
# Save the plot and display it so that is is visible on Github
p.output_backend = "svg"
export_svgs(p, filename="airline_retrain.svg")
display(SVG('airline_retrain.svg'))

In [64] used -4538.9531 MiB RAM in 1.87s, total RAM usage 42944.00 MiB


As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.